In [ ]:
# Import moduls
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


from math import nan

from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.pipeline import Pipeline 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
import keras.losses as kl
from tensorflow.keras.callbacks import EarlyStopping

from core.classes import FeatureEngineer, DataImputation

pd.set_option('display.max_columns', None)
plt.style.use('bmh')

import statsmodels
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose


import warnings
# skforecast
import skforecast
from skforecast.datasets import fetch_dataset
from skforecast.plot import set_dark_theme, plot_prediction_intervals
from skforecast.stats import Sarimax, Arima
from skforecast.recursive import ForecasterStats
from skforecast.utils import expand_index
from skforecast.model_selection import TimeSeriesFold, backtesting_stats, grid_search_stats


In [ ]:
path = '../data/processed/combined_data_with_elasticity.csv'
df = pd.read_csv(path, index_col='date', skipfooter=1, parse_dates=True)
size_diagrams = (10,3)


## Classes for training the models

In [ ]:
class Model(TransformerMixin, BaseEstimator):
    """Imputs missing values. """
    
    def __init__(self,columns, model_start, model_middle, model_end): 
        """Initalizes the attributes."""
        self.features = []
        self.num_cols = []
        self.dict_num = {}
        self.model_start = model_start
        self.model_middle = model_middle 
        self.model_end = model_end
        self.columns = columns
        self.weight_mond = 0
        self.weight_fer = 0
        self.weight_mond_fer = 0
        self.weight_fer_fer = 0
        self.weight_lindt = 0

    def fit(self, X, y=None):
        mask_f = X.loc[:,'Ferrero'].isna()
        mask_l = X.loc[:,'LindtSpruengli'].isna()
        number_mond = (~X['Mondelez'].isna()).sum()
        number_fer = (~X['Ferrero'].isna()).sum()
        number_lindt = (~X['LindtSpruengli'].isna()).sum()
        number_all = number_mond + number_fer + number_lindt
        self.weight_mond_fer = number_mond / (number_mond + number_fer)
        self.weight_fer_fer = number_fer / (number_mond + number_fer)
        self.weight_mond = number_mond / number_all
        self.weight_fer = number_fer / number_all
        self.weight_lindt = number_lindt / number_all
        feat_start = X
        target_start = y
        self.model_start.fit(feat_start.loc[:,self.columns + ['Mondelez']], target_start)

        feat_middle = X.copy()
        feat_middle = feat_middle.loc[~mask_f,:]
        target_middle = y.loc[~mask_f]
        feat_middle['target_pred_by_1'] = self.model_start.predict(feat_middle.loc[:,self.columns + ['Mondelez']])

        self.model_middle.fit(feat_middle.loc[:,['target_pred_by_1', 'Ferrero']], target_middle)

        feat_end = X.copy()
        feat_end = feat_end.loc[~mask_f & ~mask_l,:]
        target_end = y.loc[~mask_f & ~mask_l]
        feat_end['target_pred_by_1'] = self.model_start.predict(feat_end.loc[:,self.columns + ['Mondelez']])
        feat_end['target_pred_by_2'] = self.model_middle.predict(feat_end.loc[:,['target_pred_by_1', 'Ferrero']])

        self.model_end.fit(feat_end.loc[:,['target_pred_by_2', 'LindtSpruengli']], target_end)

        return self
    
    def transform(self, X, y=None):
        """Fill the missing values and drop some columns.
        
        
        """
        df = X.copy()

        self.features = df.columns.values
        return df
    
    def predict(self, X, y=None):
        """Fill the missing values and drop some columns.
        
        
        """
        mask_f = X.loc[:,'Ferrero'].isna()
        mask_l = X.loc[:,'LindtSpruengli'].isna()
        result = pd.DataFrame.from_dict({'index': X.index.values})
        result = result.set_index('index')
        result['pred_m'] = nan
        result['pred_mf'] = nan
        result['pred_mfl'] = nan
        feat_middle = X.copy()
        feat_middle = feat_middle.loc[~mask_f,:]
        feat_middle['target_pred_by_1'] = self.model_start.predict(feat_middle.loc[:,self.columns + ['Mondelez']])
        result.loc[:,'pred_m'] = self.model_start.predict(X.loc[:,self.columns + ['Mondelez']])
       
        feat_end = X.copy()
        feat_end = feat_end.loc[~mask_f & ~mask_l,:]
        feat_end['target_pred_by_1'] = self.model_start.predict(feat_end.loc[:,self.columns + ['Mondelez']])
        feat_end['target_pred_by_2'] = self.model_middle.predict(feat_end.loc[:,['target_pred_by_1', 'Ferrero']])
        result.loc[~mask_f,'pred_mf'] = self.model_middle.predict(feat_middle.loc[:,['target_pred_by_1', 'Ferrero']])
        result.loc[~mask_f & ~mask_l,'pred_mfl'] = self.model_end.predict(feat_end.loc[:,['target_pred_by_2', 'LindtSpruengli']])
        result['final'] = 0
        result.loc[mask_f, 'final'] = result.loc[mask_f,'pred_m'] 
        result.loc[~mask_f & mask_l,'final'] = result.loc[~mask_f & mask_l,'pred_m'] * self.weight_mond_fer + result.loc[~mask_f & mask_l,'pred_mf'] * self.weight_fer_fer 
        result.loc[~mask_l,'final'] = result.loc[~mask_l,'pred_m'] * self.weight_mond + result.loc[~mask_l,'pred_mf'] * self.weight_fer + result.loc[~mask_l,'pred_mfl'] * self.weight_lindt
        return result
    
    def get_feature_names_out(self, input_features=None):
        """Returns the names of the features."""
        return self.features

In [ ]:
class Model_pred_error(TransformerMixin, BaseEstimator):
    """Imputs missing values. """
    
    def __init__(self, columns, model_start, model_middle, model_end): 
        """Initalizes the attributes."""
        self.features = []
        self.num_cols = []
        self.dict_num = {}
        self.model_start = model_start
        self.model_middle = model_middle 
        self.model_end = model_end
        self.columns = columns
        self.weight_mond = 0
        self.weight_fer = 0
        self.weight_mond_fer = 0
        self.weight_fer_fer = 0
        self.weight_lindt = 0
        self.eta = 0.9

    def fit(self, X, y=None):
        mask_f = X.loc[:,'Ferrero'].isna()
        mask_l = X.loc[:,'LindtSpruengli'].isna()
        number_mond = (~X['Mondelez'].isna()).sum()
        number_fer = (~X['Ferrero'].isna()).sum()
        number_lindt = (~X['LindtSpruengli'].isna()).sum()
        number_all = number_mond + number_fer + number_lindt
        self.weight_mond_fer = number_mond / (number_mond + number_fer)
        self.weight_fer_fer = number_fer / (number_mond + number_fer)
        self.weight_mond = number_mond / number_all
        self.weight_fer = number_fer / number_all
        self.weight_lindt = number_lindt / number_all
        feat_start = X
        target_start = y
        self.model_start.fit(feat_start.loc[:,self.columns + ['Mondelez']], target_start)
        y_start_predicted = self.model_start.predict(feat_start.loc[:,self.columns + ['Mondelez']])
        
        feat_middle = X.copy()
        target_middle = (y-self.eta*y_start_predicted).loc[~mask_f]
        
        self.model_middle.fit(feat_middle.loc[~mask_f,self.columns + ['Mondelez', 'Ferrero']], target_middle)
        y_middle_predicted = self.model_middle.predict(feat_middle.loc[~mask_f & ~mask_l,self.columns + ['Mondelez', 'Ferrero']])
        feat_end = X.copy()
        feat_end = feat_end.loc[~mask_f & ~mask_l,:]
        target_end =( y-self.eta*y_start_predicted).loc[~mask_f & ~mask_l] 
        target_end = target_end - self.eta**2 * y_middle_predicted

        self.model_end.fit(feat_end.loc[:,self.columns + ['Mondelez', 'Ferrero', 'LindtSpruengli']], target_end)

        return self
    
    def transform(self, X, y=None):
        """Fill the missing values and drop some columns.
        
        
        """
        df = X.copy()

        self.features = df.columns.values
        return df
    
    def predict(self, X, y=None):
        """Fill the missing values and drop some columns.
        
        
        """
        mask_f = X.loc[:,'Ferrero'].isna()
        mask_l = X.loc[:,'LindtSpruengli'].isna()

        result = pd.DataFrame.from_dict({'index': X.index.values})
        result = result.set_index('index')
        result['pred_m'] = nan
        result['pred_mf'] = nan
        result['pred_mfl'] = nan
        feat_middle = X.copy()

        result['pred_m'] = self.model_start.predict(feat_middle.loc[:,self.columns + ['Mondelez']])

        result.loc[~mask_f ,'pred_mf'] = self.model_middle.predict(feat_middle.loc[~mask_f, self.columns + ['Mondelez', 'Ferrero']])
        result.loc[~mask_f & ~mask_l ,'pred_mfl'] = self.model_end.predict(feat_middle.loc[~mask_f & ~mask_l,  self.columns + ['Mondelez', 'Ferrero', 'LindtSpruengli']])
        result['final'] = 0
        result.loc[mask_f,'final'] = result.loc[mask_f,'pred_m'] 
        result.loc[~mask_f & mask_l,'final'] = result.loc[~mask_f & mask_l,'pred_m'] * self.eta + result.loc[~mask_f & mask_l,'pred_mf'] 
        result.loc[~mask_l,'final'] = result.loc[~mask_l,'pred_m'] * self.eta + result.loc[~mask_l,'pred_mf'] * self.eta**2 + result.loc[~mask_l,'pred_mfl']
        return result
    
    def get_feature_names_out(self, input_features=None):
        """Returns the names of the features."""
        return self.features

In [ ]:
def preprocessing_split(df, dict_lag, number_train, target_name):
    shares = ['Ferrero', 'LindtSpruengli', 'Mondelez']
    columns = [x for x in list(dict_lag) if x not in shares]
    col_with_target = columns
    if target_name not in columns: 
        col_with_target.append(target_name)
    df_selected_col = df.loc[:,shares + col_with_target]
    df_selected_col = df_selected_col.dropna(subset=target_name)
    df_train = df_selected_col.iloc[:number_train,:]
    df_test = df_selected_col.iloc[number_train:,:]
    target_train = df_train[target_name]
    target_test = df_test[target_name]
    features_train = df_train.loc[:,shares + columns]
    features_test = df_test.loc[:,shares + columns]
    return shares, columns, target_train, target_test, features_train, features_test 

def preprocessing(df, dict_lag, number_train, target_name):
    shares, columns, target_train, target_test, features_train, features_test  = preprocessing_split(df, dict_lag, number_train, target_name)
    ohe_tranf = ColumnTransformer(transformers = [('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), 
                                                  ['month'])], 
                                  remainder = 'passthrough', verbose_feature_names_out = False)
    pipeline = Pipeline([('DaIm', DataImputation(shares)), 
                               ('FE', FeatureEngineer(dict_lag)),
                               ('ohe', ohe_tranf),
                               ('sca', StandardScaler())])
    
    features_train_transf = pipeline.fit_transform(features_train)
    features_test_transf = pipeline.transform(features_test)
    # make the features into a dataframe again
    features_train_transf = pd.DataFrame(columns=pipeline.get_feature_names_out(), data=features_train_transf)
    features_train_transf.index = features_train.index
    features_test_transf = pd.DataFrame(columns=pipeline.get_feature_names_out(), data=features_test_transf)
    features_test_transf.index = features_test.index
    columns = [x for x in pipeline.get_feature_names_out() if x not in shares]
    return features_train_transf, features_test_transf, target_train, target_test, columns

In [ ]:
def train_model_combination(df, dict_lag, model_start, model_middle, model_end, number_train, target_name, name='Model', model_simple=0):
    # Train Test split
    shares, columns, target_train, target_test, features_train, features_test = preprocessing_split(df, dict_lag, number_train, target_name)
    ohe_tranf = ColumnTransformer(transformers = [('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), 
                                                  ['month'])], 
                                  remainder = 'passthrough', verbose_feature_names_out=False)
    pipeline = Pipeline([('DaIm', DataImputation(shares)), 
                               ('FE', FeatureEngineer(dict_lag)),
                               ('ohe', ohe_tranf),
                               ('sca', StandardScaler())])
    
    features_train_transf = pipeline.fit_transform(features_train)
    features_test_transf = pipeline.transform(features_test)
    # make the features into a dataframe again
    features_train_transf = pd.DataFrame(columns=pipeline.get_feature_names_out(), data=features_train_transf)
    features_train_transf.index = features_train.index
    features_test_transf = pd.DataFrame(columns=pipeline.get_feature_names_out(), data=features_test_transf)
    features_test_transf.index = features_test.index
    columns = [x for x in pipeline.get_feature_names_out() if x not in shares]
    # Model
    if model_simple == 0:
        mod = Model( columns, model_start, 
                    model_middle, model_end)
    else: 
        mod = Model_pred_error(columns, model_start, 
                   model_middle, model_end) 
    mod.fit(features_train_transf, target_train)

    #Predict Train data
    results = mod.predict(features_train_transf, target_train)
    mse_train = mean_squared_error(results['final'], target_train)
    print('MSE on Train set: ', mse_train)
    results['target'] = target_train
    last_train_df = results.iloc[-1,:]
    last_train_df = pd.DataFrame(last_train_df).transpose()
    results_test = mod.predict(features_test_transf, target_test)
    results_test['target'] = target_test
    mse_test = mean_squared_error(results_test['final'], target_test)
    print('MSE on Test set: ', mse_test)
    results_test = pd.concat([last_train_df, results_test])
    sns.set(font_scale=1)
    color_target = 'darkred'
    color_final = 'darkgreen'
    color_m = 'orchid'
    color_mf = 'lightblue'
    color_mfl = 'pink'
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=size_diagrams)
    ax.plot(results.index, results.pred_m.values, label='Hilfsmodell 1', alpha=0.5, color=color_m)
    ax.plot(results.index, results.pred_mf.values, label='Hilfsmodell 2', alpha=0.5, color=color_mf)
    ax.plot(results.index, results.pred_mfl.values, label='Hilfsmodell 3', alpha=0.5, color=color_mfl)
    ax.plot(results.index, results.final.values, label='Finales Modell', color=color_final)
    ax.plot(results.index, results.target, label='Target', color=color_target);
    ax.set_ylabel('Price elasticity', alpha=0.8)
    ax.set_title(name)
    
    # Predict testdata 
    ax.plot(results_test.index, results_test.target, color=color_target, linestyle='--')
    
    ax.plot(results_test.index, results_test.pred_m.values, alpha=0.5, linestyle='--', color=color_m)
    ax.plot(results_test.index, results_test.pred_mf.values, alpha=0.5, linestyle='--', color=color_mf)
    ax.plot(results_test.index, results_test.pred_mfl.values, alpha=0.5, linestyle='--', color=color_mfl)
    ax.plot(results_test.index, results_test.final.values,  linestyle='--', color=color_final)
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    ylim_min = ax.get_ylim()[0]
    ylim_max = ax.get_ylim()[1]
    laenge = ylim_max - ylim_min 
    laenge_neu = laenge * 0.9 
    laenge_neu2 = laenge_neu * 0.9
    xmax = ax.get_xlim()[1]
    ax.fill_betweenx(ylim, pd.to_datetime('2021-04-30'), xmax, alpha=0.1, color='black')
    ax.fill_betweenx([laenge_neu * ylim_min / laenge , laenge_neu * ylim_max / laenge], pd.to_datetime('2024-11-30'), xmax, alpha=0.1, color='black')
    ax.fill_betweenx([laenge_neu2 * ylim_min / laenge , laenge_neu2 * ylim_max / laenge], pd.to_datetime('2025-02-28'), xmax, alpha=0.1, color='black', hatch='xx', label='Train data')
    ax.text(x=pd.to_datetime('2019-02-05'), y=ylim[0], s='Mondelez', fontsize='x-large', alpha=0.2)
    ax.text(x=pd.to_datetime('2021-05-05'), y=ylim[0], s='Ferrero', fontsize='x-large', alpha=0.2)
    ax.text(x=pd.to_datetime('2024-12-02'), y=laenge_neu * ylim_min / laenge, s='Lindt', fontsize='medium', alpha=0.2)
    ax.set_xlim([pd.to_datetime('2019-01-01'), pd.to_datetime('2026-01-01') ])
    ax.legend()
    return results, results_test, mse_train, mse_test, mod, pipeline
    

In [ ]:
def feature_importance(model, pipeline, df, dict_lag, number_train, target_name, predict_typ='normal'):
    """Calculate the importance of so features from the dataset and the engineered features and plot them.
    
    Args:
        pipeline (pipeline): Pipeline with the following steps as feature engineering: 'daTr', 'daIm', 'FE', 'm_daIm'
                             and afterwards the following steps doing the predictions: 'preprocessor', 'model'
    
    Returns:
        dict: Feature importance of the original features, given as the values of the dict
        dict: Feature importance of the engineered features, given as the values of the dict
    
    """
    shares, columns, target_train, target_test, features_train, features_test = preprocessing_split(df, dict_lag, number_train, target_name)
    features_train_transf = pipeline.transform(features_train)
    features_test_transf = pipeline.transform(features_test)
    # make the features into a dataframe again
    features_train_transf = pd.DataFrame(columns=pipeline.get_feature_names_out(), data=features_train_transf)
    features_train_transf.index = features_train.index
    features_test_transf = pd.DataFrame(columns=pipeline.get_feature_names_out(), data=features_test_transf)
    features_test_transf.index = features_test.index
    columns = [x for x in pipeline.get_feature_names_out() if x not in shares]
    if predict_typ == 'stat': 
        scor_orig = mean_squared_error(target_test.values.tolist(), model_res.get_forecast(steps=len(target_test), exog = features_test_transf).predicted_mean)
    elif predict_typ == 'normal':
        scor_orig = mean_squared_error(target_test.values.tolist(), model.predict(features_test_transf).final)
    else:
        scor_orig = mean_squared_error(target_test.values.tolist(), model.predict(features_test_transf))
    
    features_test_perm = features_test.copy()
    columns = features_test_perm.columns.values
    for i in range(2, len(columns)):
        col = columns[i]
        try:
            age_series_perm = features_test_perm[col].sample(frac=1, replace=False, random_state=i+1)
            age_series_perm = age_series_perm.reset_index(drop=True).tolist()      
            features_test_perm[col] = age_series_perm
        except: 
            print('Spalte nicht vorhanden')
    features_test_transf_perm = pipeline.transform(features_test_perm)
    features_test_transf_perm = pd.DataFrame(columns=pipeline.get_feature_names_out(), data=features_test_transf_perm)
    features_test_transf_perm.index = features_test.index
    if predict_typ == 'stat': 
        vgl = mean_squared_error(target_test.values.tolist(), model_res.get_forecast(steps=len(target_test), exog = features_test_transf_perm).predicted_mean)
    elif predict_typ == 'normal':
        vgl = mean_squared_error(target_test.values.tolist(), model.predict(features_test_transf_perm).final)
    else:
        vgl = mean_squared_error(target_test.values.tolist(), model.predict(features_test_transf_perm))
  
    features_test_perm = features_test.copy()
    columns = features_test_perm.columns.values
    results_FI_start = pd.DataFrame([np.full(len(['Model'] + list(dict_lag.keys())), float('nan'))], columns=['Model'] + list(dict_lag.keys()))  
    for i in range(0, len(columns)):
        col = columns[i]
        features_test_perm = features_test.copy()
        age_series_perm = features_test_perm[col].sample(frac=1, replace=False, random_state=i+1)
        age_series_perm = age_series_perm.reset_index(drop=True).tolist()    
        features_test_perm[col] = age_series_perm
        features_test_transf_perm = pipeline.transform(features_test_perm)
        features_test_transf_perm = pd.DataFrame(columns=pipeline.get_feature_names_out(), data=features_test_transf_perm)
        features_test_transf_perm.index = features_test.index
        if predict_typ == 'stat': 
            score = mean_squared_error(target_test.values.tolist(), model_res.get_forecast(steps=len(target_test), exog = features_test_transf_perm).predicted_mean)
        elif predict_typ == 'normal':
            score = mean_squared_error(target_test.values.tolist(), model.predict(features_test_transf_perm).final)
        else:
            score = mean_squared_error(target_test.values.tolist(), model.predict(features_test_transf_perm))
        results_FI_start.loc[0,col] = score-scor_orig

    columns_FE = features_test_transf.columns.values.tolist()
    results_FI_end = pd.DataFrame([np.full(len(['Model'] + columns_FE), float('nan'))], columns=['Model'] + columns_FE)
    for i in range(len(columns_FE)):
        col = columns_FE[i]
        # Copy data
        features_test_perm = features_test_transf.copy()
        # Shuffle one column
        age_series_perm = features_test_perm[col].sample(frac=1, replace=False, random_state=0)
        age_series_perm = age_series_perm.reset_index(drop=True).tolist()
        features_test_perm[col] = age_series_perm
        if predict_typ == 'stat': 
            score = mean_squared_error(target_test.values.tolist(), model_res.get_forecast(steps=len(target_test), exog = features_test_perm).predicted_mean)
        elif predict_typ == 'normal':
            score = mean_squared_error(target_test.values.tolist(), model.predict(features_test_perm).final)
        else:
            score = mean_squared_error(target_test.values.tolist(), model.predict(features_test_perm))
        results_FI_end.loc[0,col] = score-scor_orig
    return results_FI_start, results_FI_end

In [ ]:
def load_data(name, number_train=74, incl_shares=True):
    if incl_shares == True:
        shares = ['Ferrero', 'LindtSpruengli', 'Mondelez']
    else:
        shares = []

    if name == 'coffee':
        target_name = 'elasticity_coffee'
        dict_lag = {}
        for k in shares + ['month']:
            dict_lag.update({k: [0]})
        for k in [
            'EinfPr_Kaffee und Tee, Kaffee-Ersatz',
            'VPI_Kaffee und Ähnliches', 
            'elasticity_coffee']:
            dict_lag.update({k: [1]})

        for k in ['ErzPr_Kaffee und Tee, Kaffee-Ersatz', 
                    'PCOCOUSDM',
                        'PCOFFROBUSDM']:
            dict_lag.update({k: [0,1]})
    elif name == 'cacao':
        columns = [
            #'ErzPr_Schokolade u.a. kakaoh. Lebensm.zub.,in Verp.>2kg',
            #'ErzPr_Schokolade u.a. kakaoh. Leb.m.zuber.,in Verp.<=2kg',
            #'ErzPr_Süßwaren oh. Kakaogeh. (einschl.weißer Schokolade)',
            'month',
            'ErzPr_Schokolade u.a. kakaoh. Leb.m.zuber.,in Verp.<=2kg',
            'EinfPr_Süßwaren (ohne Dauerbackwaren)',
            'VPI_Schokoladen', 
            #'VPI_Süßwaren',
            #'VPI_Kakaopulver oder Ähnliches',
            'PCOCOUSDM',
            'PCOFFROBUSDM']
        target_name = 'elasticity_cacao'
        dict_lag = {}
        for k in shares:
            dict_lag.update({k: [0]})
            dict_lag.update({'Mondelez': [0,1,2,3]})
        for k in [ #'Umsatz_WZ08-1082',
            #'Wert der zum Absatz bestimmten Produktion_Schokolade u.a. kakaohaltige Lebensmittelzubereit.',
            'EinfPr_Süßwaren (ohne Dauerbackwaren)',
            'VPI_Schokoladen',
            #'VPI_Süßwaren',
            #'VPI_Kakaopulver oder Ähnliches',
            'PCOCOUSDM',
            'PCOFFROBUSDM', 
            'elasticity_cacao']:
            dict_lag.update({k: [1]})

        for k in [
            #'ErzPr_Schokolade u.a. kakaoh. Lebensm.zub.,in Verp.>2kg',
            #'ErzPr_Schokolade u.a. kakaoh. Leb.m.zuber.,in Verp.<=2kg',
            #'ErzPr_Süßwaren oh. Kakaogeh. (einschl.weißer Schokolade)',
            'month',
            'ErzPr_Schokolade u.a. kakaoh. Leb.m.zuber.,in Verp.<=2kg', 
            'PCOCOUSDM',
            'PCOFFROBUSDM']:
            dict_lag.update({k: [0,1]})
    else: 
        print('Only "coffee" and "cacao" are valid entries.')
        return -1
    return target_name, number_train, shares, dict_lag

In [ ]:
def run_both_models(name_start, df, dict_lag, model_start, model_middle, model_end, number_train, 
                    target_name, results_combined, results_start, results_end):
    for i in range(2):
        if i == 0:
            name = name_start + ' Boosting'
        else:
            name = name_start +  ' Boosting (errors)'
        results, results_test, mse_train, mse_test, mod1, pipeline1 = train_model_combination(df, dict_lag, model_start, model_middle, model_end, number_train, target_name, name, model_simple=i)
        
        results_new = pd.DataFrame([[name, name_start, i, mse_train, mse_test, mod1, pipeline1]], columns=columns_results)
        # Save MSE 
        results_combined = pd.concat([results_combined, results_new])
        results_combined = results_combined.reset_index(drop=True)

        results_FI_start, results_FI_end = feature_importance(mod1, pipeline1, df, dict_lag, number_train, target_name)
        results_FI_start.loc[0,'Model'] = name
        results_FI_end.loc[0,'Model'] = name
        results_start = pd.concat([results_start, results_FI_start])
        results_end = pd.concat([results_end, results_FI_end])
        results_start = results_start.reset_index(drop=True)
        results_end = results_end.reset_index(drop=True)
    return results_combined, results_start, results_end 

## Cacao elasticity

In [ ]:
target_name, number_train, shares, dict_lag = load_data('cacao')
results_start_cacao = pd.DataFrame([], columns=['Model'] + list(dict_lag.keys()))

features_train_transf, features_test_transf, target_train, target_test, columns = preprocessing(df, dict_lag, 
                                                                                                number_train, target_name)
columns_FE = features_test_transf.columns.values.tolist()
results_end_cacao = pd.DataFrame([], columns=['Model'] + columns_FE)

columns_results = ['Model_name', 'Model_typ', 'Variant', 'MSE Train', 'MSE Test', 'Model', 'Pipeline']
results_combined_cacao = pd.DataFrame(columns = columns_results)

In [ ]:
corr = df.loc[:,list(dict_lag)].corr()
fig_heatmap, ax_heatmap = plt.subplots(nrows=1, ncols=1, figsize=(10,10));
sns.heatmap(corr.abs(), xticklabels=True, yticklabels=True, ax=ax_heatmap)

## Linear Regression

In [ ]:
model_start = LinearRegression()
model_middle = LinearRegression()
model_end = LinearRegression()

name = 'Linear Regression'
results_combined_cacao, results_start_cacao, results_end_cacao = run_both_models(name, df, dict_lag, model_start, model_middle, model_end, number_train, target_name, results_combined_cacao, results_start_cacao, results_end_cacao)

## LinearSVR

In [ ]:
model_start = LinearSVR()
model_middle = LinearSVR()
model_end = LinearSVR()

name = 'Linear SVR'
results_combined_cacao, results_start_cacao, results_end_cacao = run_both_models(name, df, dict_lag, model_start, model_middle, model_end, number_train, target_name, results_combined_cacao, results_start_cacao, results_end_cacao)

In [ ]:
model_start = SVR()
model_middle = SVR()
model_end = SVR()

name = 'SVR'
results_combined_cacao, results_start_cacao, results_end_cacao = run_both_models(name, df, dict_lag, model_start, model_middle, model_end, number_train, target_name, results_combined_cacao, results_start_cacao, results_end_cacao)

## Random Forest

In [ ]:
model_start = RandomForestRegressor(n_estimators=5, max_depth=5)
model_middle = RandomForestRegressor(n_estimators=5, max_depth=5)
model_end = RandomForestRegressor(n_estimators=1, max_depth=5)

name = 'Random Forest Regressor'
results_combined_cacao, results_start_cacao, results_end_cacao = run_both_models(name, df, dict_lag, model_start, model_middle, model_end, number_train, target_name, results_combined_cacao, results_start_cacao, results_end_cacao)

### ANN

In [ ]:
target_name, number_train, shares, dict_lag = load_data('cacao', incl_shares=False)

In [ ]:
shares, columns, target_train, target_test, features_train, features_test = preprocessing_split(df, dict_lag, number_train, target_name)
ohe_tranf = ColumnTransformer(transformers = [('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), 
                                              ['month'])], 
                              remainder = 'passthrough', verbose_feature_names_out=False)
pipeline_ann = Pipeline([('DaIm', DataImputation(shares)), 
                               ('FE', FeatureEngineer(dict_lag)),
                               ('ohe', ohe_tranf),
                               ('sca', StandardScaler())])
    
features_train_transf = pipeline_ann.fit_transform(features_train)
features_test_transf = pipeline_ann.transform(features_test)
# make the features into a dataframe again
features_train_transf = pd.DataFrame(columns=pipeline_ann.get_feature_names_out(), data=features_train_transf)
features_train_transf.index=features_train.index
features_test_transf = pd.DataFrame(columns=pipeline_ann.get_feature_names_out(), data=features_test_transf)
features_test_transf.index=features_test.index
columns = [x for x in pipeline_ann.get_feature_names_out() if x not in shares]

In [ ]:
model_an = Sequential()
model_an.add(Dense(50, activation="relu", input_shape=(features_train_transf.shape[1],)))
#model_an.add(Dense(500, activation = 'relu'))
model_an.add(Dense(20, activation = 'relu'))
#model_an.add(Dropout(rate=0.3))
model_an.add(Dense(10, activation = 'relu'))
#model_an.add(Dropout(rate=0.3))
#model_an.add(Dense(10, activation = 'relu'))
#model_an.add(Dropout(rate=0.3))
model_an.add(Dense(1))

model_an.compile(loss=tf.keras.losses.mse,
                optimizer='adam',
                #metrics = ["mae"]
                )  # compile the model

early_stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10)
hist = model_an.fit(features_train_transf, 
                    target_train, 
                    validation_data=(features_test_transf, target_test),
                    epochs=25, 
                    batch_size=1,
                    callbacks=[early_stop]
                   )#fits the model
# batch_size: 32 bis 512, also 32, 64, 128, 256, 512


fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 5.5))
l = len(hist.history['loss'])
ax[0].plot(range(l), hist.history['loss'], linestyle=':', color='orange', label='Loss')
ax[0].plot(range(l), hist.history['val_loss'], linestyle=':', color='green', label='validation loss')
ax[0].legend()

In [ ]:
# Predictions on train set
results = model_an.predict(features_train_transf)
mse_train = mean_squared_error(results, target_train)
print('MSE on Train set: ', mse_train)
# Plot predictions on train set
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=size_diagrams)
color_target = 'darkred'
color_ann = 'darkgreen'
ax.plot(target_train.index, results, label='ANN', color=color_ann);
ax.plot(target_train.index, target_train,  label='Target', color=color_target);
#Predictions on test set
results_test = model_an.predict(features_test_transf)
mse_test = mean_squared_error(results_test, target_test)
print('MSE on Test set: ', mse_test)
# Save MSE 
results_new = pd.DataFrame([['ANN', 'ANN', -1, mse_train, mse_test, model_an, pipeline_ann]], columns=columns_results)
results_combined_cacao = pd.concat([results_combined_cacao, results_new])
results_combined_cacao = results_combined_cacao.reset_index(drop=True)
# Plot preditions on test set
value = results[-1]
value = value.tolist()[0]
results_list = results_test.tolist()
results_list.insert(0,[value])

value_date = target_train.index[-1]
new_index = target_test.index
new_index= new_index.tolist()
new_index.insert(0,value_date)
ax.plot(new_index, results_list , color=color_ann, linestyle='--');
ax.plot(new_index, [target_train.values.tolist()[-1]] + target_test.values.tolist(), color=color_target, linestyle='--');
xlim = ax.get_xlim()
ylim = ax.get_ylim()
ylim_min = ax.get_ylim()[0]
ylim_max = ax.get_ylim()[1]
laenge = ylim_max - ylim_min 
laenge_neu = laenge* 0.9 
laenge_neu2 = laenge_neu 
xmax = ax.get_xlim()[1]
ax.fill_betweenx([laenge_neu2 * ylim_min / laenge , laenge_neu2 * ylim_max / laenge], pd.to_datetime('2025-02-28'), xmax, alpha=0.1, color='black', hatch='xx', label='Train data')
ax.set_xlim([pd.to_datetime('2019-01-01'), pd.to_datetime('2026-01-01') ])
ax.set_ylabel('Price elasticity', alpha=0.8)
ax.set_title('ANN')
ax.legend()

In [ ]:
results_FI_start, results_FI_end = feature_importance(model_an, pipeline_ann, df, dict_lag,number_train, target_name, predict_typ='ann')
results_FI_start.loc[0, 'Model'] = 'ANN'
results_FI_end.loc[0, 'Model'] = 'ANN'
results_start_cacao  = pd.concat([results_start_cacao, results_FI_start])
results_end_cacao = pd.concat([results_end_cacao, results_FI_end])
results_start_cacao = results_start_cacao.reset_index(drop=True)
results_end_cacao = results_end_cacao.reset_index(drop=True)

In [ ]:
results_combined_cacao

## Coffee elasticity

In [ ]:
# Initialize values
target_name, number_train, shares, dict_lag = load_data('coffee')
results_start = pd.DataFrame([], columns=['Model'] + list(dict_lag.keys()))

features_train_transf, features_test_transf, target_train, target_test, columns = preprocessing(df, dict_lag, 
                                                                                                number_train, target_name)
columns_FE = features_test_transf.columns.values.tolist()
results_end = pd.DataFrame([], columns=['Model'] + columns_FE)

columns_results = ['Model_name', 'Model_typ', 'Variant', 'MSE Train', 'MSE Test', 'Model', 'Pipeline']
results_combined = pd.DataFrame(columns = columns_results)
    

In [ ]:
df_reduced = df.loc[:,list(dict_lag)]
myFE = FeatureEngineer(dict_lag)
    
df_en = myFE.fit_transform(df_reduced)
corr = df_en.corr()
fig_heatmap, ax_heatmap = plt.subplots(nrows=1, ncols=1, figsize=(10,10));
sns.heatmap(corr.abs(), xticklabels=True, yticklabels=True, ax=ax_heatmap)

In [ ]:
df_en = df.loc[:,['EinfPr_Kaffee und Tee, Kaffee-Ersatz', 'ErzPr_Kaffee und Tee, Kaffee-Ersatz', 
                   'VPI_Kaffee und Ähnliches', 'Ferrero', 'LindtSpruengli', 'Mondelez', 
                   'PCOCOUSDM', 'PCOFFROBUSDM', 'elasticity_coffee']]
corr = df_en.corr()
fig_heatmap, ax_heatmap = plt.subplots(nrows=1, ncols=1, figsize=(10,10));
sns.heatmap(corr.abs(), xticklabels=True, yticklabels=True, ax=ax_heatmap)
sns.set(font_scale=4)


### Linear Regression

In [ ]:
model_start = LinearRegression()
model_middle = LinearRegression()
model_end = LinearRegression()

name = 'Linear Regression'
results_combined, results_start, results_end = run_both_models(name, df, dict_lag, model_start, model_middle, model_end, number_train, target_name, results_combined, results_start, results_end)

### Linear SVR


In [ ]:
model_start = LinearSVR()
model_middle = LinearSVR()
model_end = LinearSVR()

name = 'Linear SVR'
results_combined, results_start, results_end = run_both_models(name, df, dict_lag, model_start, model_middle, model_end, number_train, target_name, results_combined, results_start, results_end)

### SVR

In [ ]:
model_start = SVR()
model_middle = SVR()
model_end = SVR()

name = 'SVR'
results_combined, results_start, results_end = run_both_models(name, df, dict_lag, model_start, model_middle, model_end, number_train, target_name, results_combined, results_start, results_end)

### Random Forest

In [ ]:
# Grid search for n_estimators in random forest 
for n_1 in range(1,10):
    for n_2 in range(1,10):
        for n_3 in range(1,10):


            model_start = RandomForestRegressor(n_estimators=n_1, max_depth=5, random_state=42)
            model_middle = RandomForestRegressor(n_estimators=n_2, max_depth=5, random_state=42)
            model_end = RandomForestRegressor(n_estimators=n_3, max_depth=5, random_state=42)

            name = 'Random Forest Regressor Boosting' + str(n_1) + str(n_2) + str(n_3)
            results_combined = run_both_models(name, df, dict_lag, model_start, model_middle, model_end, number_train, target_name, results_combined)


In [ ]:
results_combined.sort_values(['MSE Test']).head(20)

In [ ]:
model_start = RandomForestRegressor(n_estimators=1, max_depth=5, random_state=42)
model_middle = RandomForestRegressor(n_estimators=3, max_depth=5, random_state=42)
model_end = RandomForestRegressor(n_estimators=8, max_depth=5, random_state=42)

name = 'Random Forest Regressor' 
results_combined, results_start, results_end = run_both_models(name, df, dict_lag, model_start, model_middle, model_end, number_train, target_name, results_combined, results_start, results_end)

In [ ]:
model_start = RandomForestRegressor(n_estimators=8, max_depth=5, random_state=42)
model_middle = RandomForestRegressor(n_estimators=5, max_depth=5, random_state=42)
model_end = RandomForestRegressor(n_estimators=1, max_depth=5, random_state=42)

name = 'Random Forest Regressor' 
results_combined, results_start, results_end = run_both_models(name, df, dict_lag, model_start, model_middle, model_end, number_train, target_name, results_combined, results_start, results_end)

### ANN

In [ ]:
target_name, number_train, shares, dict_lag = load_data('coffee', incl_shares=False)

In [ ]:
shares, columns, target_train, target_test, features_train, features_test = preprocessing_split(df, dict_lag, number_train, target_name)
ohe_tranf = ColumnTransformer(transformers = [('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), 
                                              ['month'])], 
                              remainder = 'passthrough', verbose_feature_names_out=False)
pipeline_ann = Pipeline([('DaIm', DataImputation(shares)), 
                               ('FE', FeatureEngineer(dict_lag)),
                               ('ohe', ohe_tranf),
                               ('sca', StandardScaler())])
    
features_train_transf = pipeline_ann.fit_transform(features_train)
features_test_transf = pipeline_ann.transform(features_test)
# make the features into a dataframe again
features_train_transf = pd.DataFrame(columns=pipeline_ann.get_feature_names_out(), data=features_train_transf)
features_train_transf.index=features_train.index
features_test_transf = pd.DataFrame(columns=pipeline_ann.get_feature_names_out(), data=features_test_transf)
features_test_transf.index=features_test.index
columns = [x for x in pipeline_ann.get_feature_names_out() if x not in shares]

In [ ]:
model_an = Sequential()
model_an.add(Dense(50, activation="relu", input_shape=(features_train_transf.shape[1],)))
#model_an.add(Dense(500, activation = 'relu'))
model_an.add(Dense(20, activation = 'relu'))
#model_an.add(Dropout(rate=0.3))
model_an.add(Dense(10, activation = 'relu'))
#model_an.add(Dropout(rate=0.3))
#model_an.add(Dense(10, activation = 'relu'))
#model_an.add(Dropout(rate=0.3))
model_an.add(Dense(1))

model_an.compile(loss=tf.keras.losses.mse,
                optimizer='adam',
                #metrics = ["mae"]
                )  # compile the model

early_stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10)
hist = model_an.fit(features_train_transf, 
                    target_train, 
                    validation_data=(features_test_transf, target_test),
                    epochs=25, 
                    batch_size=1,
                    callbacks=[early_stop]
                   )#fits the model
# batch_size: 32 bis 512, also 32, 64, 128, 256, 512


fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 5.5))
l = len(hist.history['loss'])
ax[0].plot(range(l), hist.history['loss'], linestyle=':', color='orange', label='Loss')
ax[0].plot(range(l), hist.history['val_loss'], linestyle=':', color='green', label='validation loss')
ax[0].legend()

In [ ]:
# Predictions on train set
results = model_an.predict(features_train_transf)
mse_train = mean_squared_error(results, target_train)
print('MSE on Train set: ', mse_train)
# Plot predictions on train set
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=size_diagrams)
color_target = 'darkred'
color_ann = 'darkgreen'
ax.plot(target_train.index, results, label='ANN', color=color_ann);
ax.plot(target_train.index, target_train,  label='Target', color=color_target);
#Predictions on test set
results_test = model_an.predict(features_test_transf)
mse_test = mean_squared_error(results_test, target_test)
print('MSE on Test set: ', mse_test)
# Save MSE 
results_new = pd.DataFrame([['ANN', 'ANN', -1, mse_train, mse_test, model_an, pipeline_ann]], columns=columns_results)
results_combined = pd.concat([results_combined, results_new])
results_combined = results_combined.reset_index(drop=True)
# Plot preditions on test set
value = results[-1]
value = value.tolist()[0]
results_list = results_test.tolist()
results_list.insert(0,[value])

value_date = target_train.index[-1]
new_index = target_test.index
new_index= new_index.tolist()
new_index.insert(0,value_date)
ax.plot(new_index, results_list , color=color_ann, linestyle='--');
ax.plot(new_index, [target_train.values.tolist()[-1]] + target_test.values.tolist(), color=color_target, linestyle='--');
xlim = ax.get_xlim()
ylim = ax.get_ylim()
ylim_min = ax.get_ylim()[0]
ylim_max = ax.get_ylim()[1]
laenge = ylim_max - ylim_min 
laenge_neu = laenge* 0.9 
laenge_neu2 = laenge_neu 
xmax = ax.get_xlim()[1]
ax.fill_betweenx([laenge_neu2 * ylim_min / laenge , laenge_neu2 * ylim_max / laenge], pd.to_datetime('2025-02-28'), xmax, alpha=0.1, color='black', hatch='xx', label='Train data')
ax.set_xlim([pd.to_datetime('2019-01-01'), pd.to_datetime('2026-01-01') ])
ax.set_ylabel('Price elasticity', alpha=0.8)
ax.set_title('ANN')
ax.legend()

In [ ]:
results_FI_start, results_FI_end = feature_importance(model_an, pipeline_ann, df, dict_lag,number_train, target_name, predict_typ='ann')
results_FI_start.loc[0, 'Model'] = 'ANN'
results_FI_end.loc[0, 'Model'] = 'ANN'
results_start = pd.concat([results_start, results_FI_start])
results_end = pd.concat([results_end, results_FI_end])
results_start = results_start.reset_index(drop=True)
results_end = results_end.reset_index(drop=True)

### ARIMAX

In [ ]:
# Ignore filtewarnings
warnings.filterwarnings('once')
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [ ]:
target_name, number_train, shares, dict_lag = load_data('coffee', incl_shares = False)

shares, columns, target_train, target_test, features_train, features_test  = preprocessing_split(df, dict_lag, number_train, target_name)
ohe_tranf = ColumnTransformer(transformers = [('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), 
                                               ['month'])], 
                              remainder = 'passthrough', verbose_feature_names_out=False)
pipeline_arimax = Pipeline([('DaIm', DataImputation(shares)), 
                                  ('FE', FeatureEngineer(dict_lag)),
                                  ('ohe', ohe_tranf),
                                  ('sca', StandardScaler())])
    
features_train_transf = pipeline_arimax.fit_transform(features_train)
features_test_transf = pipeline_arimax.transform(features_test)
# make the features into a dataframe again
features_train_transf = pd.DataFrame(columns=pipeline_arimax.get_feature_names_out(), data=features_train_transf)
features_train_transf.index = features_train.index
features_test_transf = pd.DataFrame(columns=pipeline_arimax.get_feature_names_out(), data=features_test_transf)
features_test_transf.index = features_test.index
columns = [x for x in pipeline_arimax.get_feature_names_out() if x not in shares]

In [ ]:
# Train-test dates
# ==============================================================================
end_train = '2025-03-31'
print(
    f"Train dates : {data.index.min()} --- {data.loc[:end_train].index.max()}  "
    f"(n={len(data.loc[:end_train])})"
)
print(
    f"Test dates  : {data.loc[end_train:].index.min()} --- {data.loc[:].index.max()}  "
    f"(n={len(data.loc[end_train:])})"
)
data_train = data.loc[:end_train]
data_test  = data.loc[end_train:]
df_train = df.loc[:end_train, :]
df_test = df.loc[end_train : , :]
features_train_transf, 
features_test_transf, 
target_train, 
target_test
# Plot
# ==============================================================================
#set_dark_theme()
fig, ax=plt.subplots(figsize=(7, 3))
ax.plot(data_train, label='train')
ax.plot(data_test, label='test')
ax.set_title('Coffee price elasticity')
ax.legend();

In [ ]:
data_diff_1 = data_train.diff().dropna()
data_diff_2 = data_diff_1.diff().dropna()

# Suppress warnings for stationarity tests
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    print('Test stationarity for original series')
    print('-------------------------------------')
    adfuller_result = adfuller(data)
    kpss_result = kpss(data)
    print(f'ADF Statistic: {adfuller_result[0]}, p-value: {adfuller_result[1]}')
    print(f'KPSS Statistic: {kpss_result[0]}, p-value: {kpss_result[1]}')

    print('\nTest stationarity for differenced series (order=1)')
    print('--------------------------------------------------')
    adfuller_result = adfuller(data_diff_1)
    kpss_result = kpss(data.diff().dropna())
    print(f'ADF Statistic: {adfuller_result[0]}, p-value: {adfuller_result[1]}')
    print(f'KPSS Statistic: {kpss_result[0]}, p-value: {kpss_result[1]}')

    print('\nTest stationarity for differenced series (order=2)')
    print('--------------------------------------------------')
    adfuller_result = adfuller(data_diff_2)
    kpss_result = kpss(data.diff().diff().dropna())
    print(f'ADF Statistic: {adfuller_result[0]}, p-value: {adfuller_result[1]}')
    print(f'KPSS Statistic: {kpss_result[0]}, p-value: {kpss_result[1]}')

# Plot series
# ==============================================================================
fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(7, 5), sharex=True)
data.plot(ax=axs[0], title='Original time series')
data_diff_1.plot(ax=axs[1], title='Differenced order 1')
data_diff_2.plot(ax=axs[2], title='Differenced order 2');

In [ ]:
# Autocorrelation plot for original and differentiated series
# ==============================================================================
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(6, 4), sharex=True)
plot_acf(data, ax=axs[0], lags=50, alpha=0.05)
axs[0].set_title('Autocorrelation original series')
plot_acf(data_diff_1, ax=axs[1], lags=50, alpha=0.05)
axs[1].set_title('Autocorrelation differentiated series (order=1)');

In [ ]:
# Partial autocorrelation plot for original and differenced series
# ==============================================================================
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(6, 3), sharex=True)
plot_pacf(data, ax=axs[0], lags=40, alpha=0.05)
axs[0].set_title('Partial autocorrelation original series')
plot_pacf(data_diff_1, ax=axs[1], lags=36, alpha=0.05)
axs[1].set_title('Partial autocorrelation differenced series (order=1)');
plt.tight_layout();

In [ ]:
# Time series decomposition of original versus differenced series
# ==============================================================================
res_decompose = seasonal_decompose(data, model='additive', extrapolate_trend='freq', period=12)
res_descompose_diff_2 = seasonal_decompose(data_diff_1, model='additive', extrapolate_trend='freq', period=12)

fig, axs = plt.subplots(nrows=4, ncols=2, figsize=(9, 6), sharex=True)

res_decompose.observed.plot(ax=axs[0, 0])
axs[0, 0].set_title('Original series', fontsize=12)
res_decompose.trend.plot(ax=axs[1, 0])
axs[1, 0].set_title('Trend', fontsize=12)
res_decompose.seasonal.plot(ax=axs[2, 0])
axs[2, 0].set_title('Seasonal', fontsize=12)
res_decompose.resid.plot(ax=axs[3, 0])
axs[3, 0].set_title('Residuals', fontsize=12)
res_descompose_diff_2.observed.plot(ax=axs[0, 1])
axs[0, 1].set_title('Differenced series (order=1)', fontsize=12)
res_descompose_diff_2.trend.plot(ax=axs[1, 1])
axs[1, 1].set_title('Trend', fontsize=12)
res_descompose_diff_2.seasonal.plot(ax=axs[2, 1])
axs[2, 1].set_title('Seasonal', fontsize=12)
res_descompose_diff_2.resid.plot(ax=axs[3, 1])
axs[3, 1].set_title('Residuals', fontsize=12)
fig.suptitle('Time series decomposition original series versus differenced series', fontsize=14)
fig.tight_layout();

### statsmodels.Sarimax

In [ ]:
# ARIMA model with statsmodels.Sarimax
# ==============================================================================
warnings.filterwarnings("ignore", category=UserWarning, message='Non-invertible|Non-stationary')
model = SARIMAX(endog = target_train, exog = features_train_transf, order = (1, 1, 1), seasonal_order = (1, 1, 1, 12))
model_res = model.fit(disp=0)
model_res.summary()

In [ ]:
# Prediction
# ==============================================================================
predictions_statsmodels = model_res.get_forecast(steps=len(target_test), exog = features_test_transf).predicted_mean
predictions_statsmodels.name = 'predictions_statsmodels'
predictions_statsmodels.index

In [ ]:
results_FI_start, results_FI_end = feature_importance(model_res, pipeline_arimax, df, dict_lag, number_train, target_name, predict_typ='stat')

results_FI_start.loc[0, 'Model'] = 'statsmodel'
results_FI_end.loc[0, 'Model'] = 'statsmodel'
results_start = pd.concat([results_start, results_FI_start])
results_end = pd.concat([results_end, results_FI_end])
results_start = results_start.reset_index(drop=True)
results_end = results_end.reset_index(drop=True)


### skforecast Sarimax

In [ ]:
# ARIMA model with skforecast Sarimax
# ==============================================================================
warnings.filterwarnings("ignore", category=UserWarning, message='Non-invertible|Non-stationary')
model = Sarimax(order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
model.fit(y=target_train, exog = features_train_transf)
model.summary()

In [ ]:
# Prediction
# ==============================================================================
predictions_skforecast_sarimax = model.predict(steps=len(target_test), exog = features_test_transf)
predictions_skforecast_sarimax = pd.DataFrame(predictions_skforecast_sarimax, index=target_test.index)
predictions_skforecast_sarimax.columns = ['skforecast']
display(predictions_skforecast_sarimax.head(4))

In [ ]:
results_FI_start, results_FI_end = feature_importance(model, pipeline_arimax, df, dict_lag, number_train, target_name, predict_typ='stat')

results_FI_start.loc[0, 'Model'] = 'sarimax'
results_FI_end.loc[0, 'Model'] = 'sarimax'
results_start = pd.concat([results_start, results_FI_start])
results_end = pd.concat([results_end, results_FI_end])
results_start = results_start.reset_index(drop=True)
results_end = results_end.reset_index(drop=True)

### skforecast ARIMA

In [ ]:
# ARIMA model with skforecast Arima
# ==============================================================================
model = Arima(order=(1, 1, 1), seasonal_order=(1, 1, 1), m=12)
model.fit(y=target_train,  suppress_warnings=True)
model.summary()

In [ ]:
# Prediction
# ==============================================================================
predictions_skforecast_arima = model.predict(steps=len(target_test))
#pred_index = expand_index(index=data_train.index, steps=len(data_test))
predictions_skforecast_arima = pd.Series(predictions_skforecast_arima, index=target_test.index)
predictions_skforecast_arima.head(4)


In [ ]:
results_FI_start, results_FI_end = feature_importance(model, pipeline_arimax, df, dict_lag, number_train, target_name, predict_typ='stat')

results_FI_start.loc[0, 'Model'] = 'arima'
results_FI_end.loc[0, 'Model'] = 'arimax'
results_start = pd.concat([results_start, results_FI_start])
results_end = pd.concat([results_end, results_FI_end])
results_start = results_start.reset_index(drop=True)
results_end = results_end.reset_index(drop=True)

In [ ]:
results_start

## Comparison of methods

In [ ]:
# Plot predictions
# ==============================================================================
fig, ax = plt.subplots(figsize=size_diagrams)
color_target = 'darkred'
color_final = 'darkgreen'
color_m = 'orchid'
color_mf = 'lightblue'
color_mfl = 'pink'

value_date = target_train.index[-1]
new_index = target_test.index
new_index= new_index.tolist()
new_index.insert(0,value_date)

value = target_train[-1]
labels = ['statsmodels', 'skforecast sarimax', 'skforecast arima']
colors = ['darkgreen', 'darkblue', 'teal']
predictions = [predictions_statsmodels, predictions_skforecast_sarimax, predictions_skforecast_arima]
for i in range(3):
    pred = predictions[i]
    results_list = pred.values.tolist()
    if i in [0,2]:
        results_list.insert(0, value)
    else: 
        results_list.insert(0, [value])
    print(results_list)
    ax.plot(new_index, results_list, label=labels[i], color=colors[i], linestyle='--')
ax.plot(new_index, [target_train.values.tolist()[-1]] + target_test.values.tolist(), color=color_target, linestyle='--');
ax.plot(target_train, label='Target', color=color_target )
xlim = ax.get_xlim()
ylim = ax.get_ylim()
ylim_min = ax.get_ylim()[0]
ylim_max = ax.get_ylim()[1]
laenge = ylim_max - ylim_min 
laenge_neu = laenge* 0.9 
laenge_neu2 = laenge_neu 
xmax = ax.get_xlim()[1]
ax.fill_betweenx([laenge_neu2 * ylim_min / laenge , laenge_neu2 * ylim_max / laenge], pd.to_datetime('2025-02-28'), xmax, alpha=0.1, color='black', hatch='xx', label='Train data')
ax.set_xlim([pd.to_datetime('2019-01-01'), pd.to_datetime('2026-01-01') ])
ax.set_ylabel('Price elasticity', alpha=0.8)
ax.set_title('Predictions with SARIMAX models')
ax.legend()

In [ ]:
results_new = pd.DataFrame([['Sarimax statsmodel', 'Sarimax', -1, 0, mean_squared_error(target_test, predictions_statsmodels.values), float('nan'), pipeline_arimax]], columns=columns_results)
results_combined = pd.concat([results_combined, results_new])

In [ ]:
results_new = pd.DataFrame([['Sarimax skforecast', 'Sarimax', -1, 0, mean_squared_error(target_test, predictions_skforecast_sarimax.skforecast.values), float('nan'), pipeline_arimax]], columns=columns_results)
results_combined = pd.concat([results_combined, results_new])

In [ ]:
results_new = pd.DataFrame([['arima skforecast', 'arima', -1, 0, mean_squared_error(target_test, predictions_skforecast_arima.values), float('nan'), pipeline_arimax]], columns=columns_results)
results_combined = pd.concat([results_combined, results_new])

## Compare different models

In [ ]:
results_combined

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5,5))
for model in results_combined['Model_typ'].unique():
    mask = results_combined['Model_typ'] == model
    ax.scatter(results_combined.loc[mask, 'MSE Train'],results_combined.loc[mask,'MSE Test'], label = model)
ylim = ax.get_ylim()
xlim = ax.get_xlim()
ax.plot(range(20), range(20), alpha = 0.2, color = 'darkred')
ax.set_ylim(0, ylim[1])
ax.set_xlim(0, xlim[1])
ax.set_xlabel('MSE on train set')
ax.set_ylabel('MSE on test set')
ax.set_title('Mean squared errors on Train and Test sets')
ax.legend()

In [ ]:
def plot_feature_importance(results_start, results_end):
    # Plot feature importance
    fig_FI, ax_FI = plt.subplots(nrows=2, ncols=1, figsize=(10,15))
    for i in range(2):
        results = [results_start, results_end][i]
        barHeight = 0.9/results.shape[0]
        br1 = np.arange(results.shape[1]-1)
        list_br = [br1]
        for model in range(results.shape[0]):
            br_new = [x + barHeight for x in list_br[-1]] 
            list_br.append(br_new)
            ax_FI[i].barh(y=br_new, height=barHeight, width=results.iloc[model, 1:], 
                        alpha=0.9,  tick_label=results.columns.values[1:], label = results.iloc[model, 0])    
            ax_FI[i].spines['bottom'].set_visible(False)
            ax_FI[i].spines['top'].set_visible(False)
            ax_FI[i].spines['right'].set_visible(False)
        ax_FI[i].legend()
        if i  == 0:
            ax_FI[i].set_title('Feature Importance of original features')
        else:
            ax_FI[i].set_title('Feature Importance of engineered features')

In [ ]:
results_start

In [ ]:
mask = (results_start['Model'] == 'Linear Regression Boosting') | (results_start['Model'] == 'ANN')| (results_start['Model'] == 'Linear SVR Boosting')


results_start_selection = results_start.loc[mask,:]
results_end_selection = results_end.loc[mask,:]

In [ ]:
plot_feature_importance(results_start_selection, results_end_selection)